## 第7章 数据规整化：清理、转换、合并、重塑


In [1]:
import numpy as np
import pandas as pd

In [7]:
'''合并数据集
    pandas.merge()'''
data1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
data2=pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
#两个DataFrame中有相同的行'a'和'b'
print(data1)
print(data2)
print(pd.merge(data1,data2,on='key')) #指定key列

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d
   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0


In [10]:
data3=pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data3':range(7)})
data4=pd.DataFrame({'rkey':['a','b','d'],'data4':range(3)})
#column name不同，可以进行分别指定
print(pd.merge(data3,data4,left_on='lkey',right_on='rkey'))

   data3 lkey  data4 rkey
0      0    b      1    b
1      1    b      1    b
2      6    b      1    b
3      2    a      0    a
4      4    a      0    a
5      5    a      0    a


In [16]:
#默认情况下，merge()做的是交集（inner），通过how属性指定。
# 多对多产生的是行的笛卡尔积，data1有3行，data2有2行，对应产生的有6行
print (pd.merge(data1,data2,how='outer'))
print('-------------------')
print (pd.merge(data1,data2,how='left'))
print('-------------------')
print (pd.merge(data1,data2,how='right'))

   data1 key  data2
0    0.0   b    1.0
1    1.0   b    1.0
2    6.0   b    1.0
3    2.0   a    0.0
4    4.0   a    0.0
5    5.0   a    0.0
6    3.0   c    NaN
7    NaN   d    2.0
-------------------
   data1 key  data2
0      0   b    1.0
1      1   b    1.0
2      2   a    0.0
3      3   c    NaN
4      4   a    0.0
5      5   a    0.0
6      6   b    1.0
-------------------
   data1 key  data2
0    0.0   b      1
1    1.0   b      1
2    6.0   b      1
3    2.0   a      0
4    4.0   a      0
5    5.0   a      0
6    NaN   d      2


In [17]:
#对于重复的列名，通过merge()的suffixes属性设置

In [17]:
L1 = ['Hello', 'World', 18, 'Apple', None]
L2 = [i.lower() for i in L1 if isinstance(i,str)]

print(L2)

['hello', 'world', 'apple']
